In [1]:
from codebase.classes import Data, Particles, ParticlesLatent
import  pystan
import argparse
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from codebase.ibis import essl, exp_and_normalise

In [2]:
exp_data = Data("1factor", 1, 50)
exp_data.generate()


In [3]:
exp_data.get_stan_data_at_t(0)

{'N': 1,
 'J': 6,
 'z': -0.5322712809013522,
 'y': array([-1.06227128, -0.0225899 , -1.82581702, -1.66613564, -1.43904415,
        -2.64936277]),
 'D': array([0, 1, 0, 0, 1, 0])}

In [4]:
log_dir = './log/debug/'

param_names = ['beta', 'alpha', 'sigma']
latent_names = ['z', 'y_latent']
particles = ParticlesLatent('normal', 1, 200,  param_names, latent_names, 12, 1)

particles.set_log_dir(log_dir)


# particles.compile_prior_model()
# particles.compile_model()
particles.load_prior_model()
# particles.load_model()

In [5]:

particles.sample_prior_particles(exp_data.get_stan_data()) # sample prior particles
particles.sample_prior_latent_variables(exp_data.get_stan_data())
particles.reset_weights()
particles.reset_latent_weights()

In [6]:
# particles.latent_particles
particles.get_latent_weights(exp_data.get_stan_data_at_t(1))
# exp_data.get_stan_data_at_t(1)

In [7]:
particles.latent_particles['y_latent'].shape

(200, 12, 6)

In [8]:
particles.resample_particles()

In [9]:
particles.sample_latent_y_star()

In [12]:
particles.latent_particles_star['z'].shape

(200,)

In [16]:
particles.particles['alpha'][:3, 0, 0]

array([2.3632687 , 5.31168826, 5.31168826])

In [17]:
particles.latent_particles['z'][:3, 0]

array([0.62021104, 2.24447457, 2.24447457])

In [9]:
## setup particles
particles.set_log_dir(log_dir)
# particles.compile_prior_model()
# particles.compile_model()
particles.load_prior_model()
particles.load_model()


In [7]:
particles.sample_prior_particles(exp_data.get_stan_data()) # sample prior particles
particles.reset_weights() # set weights to 0

log_lklhds = np.empty(exp_data.size)
degeneracy_limit = 0.5

for t in tqdm(range(exp_data.size)):
    log_incr_weights = particles.get_incremental_weights(
        exp_data.get_stan_data_at_t(t)
    )
    log_lklhds[t] =  particles.get_loglikelihood_estimate()
    particles.update_weights()
    
    if (essl(particles.weights) < degeneracy_limit * particles.size) and (t+1) < exp_data.size:
#         print("Deg %d"%(t))
        particles.resample_particles()
        particles.jitter(exp_data.get_stan_data_upto_t(t+1))

        particles.reset_weights()
    else:
        particles.update_weights()
#     save_obj(particles, 'particles%s'%(t+1), log_dir)
    



In [9]:
from scipy.special import logsumexp
save_obj(log_lklhds, 'log_lklhds', log_dir)
marg_lklhd = np.exp(logsumexp(log_lklhds))
print('Marginal Likelihood %.5f'%marg_lklhd)

Marginal Likelihood 0.00051


In [10]:
for name in ['alpha', 'Marg_cov']:
    samples = np.squeeze(particles.particles[name])
    w = exp_and_normalise(particles.weights)
    print('\n\nEstimate')
    print(np.round(np.average(samples,axis=0, weights=w),2))
    print('\nRead Data')
    print(exp_data.raw_data[name])



Estimate
[ 0.86  2.32 -0.25 -2.06 -0.33  4.64]

Read Data
[ 1.  2. -1. -2.  0.  4.]


Estimate
[[ 1.13 -0.13  0.08  0.11  0.07 -0.11]
 [-0.13  4.31  0.11  0.58  0.51  1.32]
 [ 0.08  0.11  9.87  0.74  0.4  -1.41]
 [ 0.11  0.58  0.74  1.28 -0.   -0.59]
 [ 0.07  0.51  0.4  -0.    4.06  1.26]
 [-0.11  1.32 -1.41 -0.59  1.26  9.58]]

Read Data
[[1. 0. 0. 0. 0. 0.]
 [0. 4. 0. 0. 0. 0.]
 [0. 0. 9. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 4. 0.]
 [0. 0. 0. 0. 0. 9.]]


In [12]:
for name in ['alpha', 'Marg_cov']:
    samples = np.squeeze(particles.particles[name])
    print('\n\nEstimate')
    print(np.round(np.mean(samples,axis=0),2))
    print('\nRead Data')
    print(exp_data.raw_data[name])



Estimate
[ 0.84  2.28 -0.23 -2.08 -0.42  4.46]

Read Data
[ 1.  2. -1. -2.  0.  4.]


Estimate
[[ 1.17 -0.23  0.13  0.07 -0.08 -0.25]
 [-0.23  4.25  0.03  0.53  0.44  0.92]
 [ 0.13  0.03 10.81  0.85  0.35 -1.5 ]
 [ 0.07  0.53  0.85  1.33 -0.04 -0.64]
 [-0.08  0.44  0.35 -0.04  3.89  0.58]
 [-0.25  0.92 -1.5  -0.64  0.58  8.1 ]]

Read Data
[[1. 0. 0. 0. 0. 0.]
 [0. 4. 0. 0. 0. 0.]
 [0. 0. 9. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 4. 0.]
 [0. 0. 0. 0. 0. 9.]]
